In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Dataset Analysis and Visualization

In [ ]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
sample_submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
sns.heatmap(train.isnull())

In [ ]:
train.describe()

In [ ]:
train.describe(include = ['O'])

In [ ]:
train.corr()

In [ ]:
sns.heatmap(train.corr())

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',data=train,hue = 'Sex')

In [ ]:
train[['Sex','Survived']].groupby(['Sex']).mean().sort_values(by = 'Survived',ascending = False)

In [ ]:
train[['Pclass','Survived']].groupby(['Pclass']).mean().sort_values(by = 'Survived',ascending = False)

In [ ]:
train[['SibSp','Survived']].groupby(['SibSp']).mean().sort_values(by = 'Survived',ascending = False)

In [ ]:
train[['Parch','Survived']].groupby(['Parch']).mean().sort_values(by = 'Survived',ascending = False)

In [ ]:
train[['Embarked','Survived']].groupby(['Embarked']).mean().sort_values(by = 'Survived',ascending = False)

In [ ]:
sns.displot(x= train['Age'].dropna(),kde=True)

In [ ]:
sns.histplot(x='Age' ,hue = 'Survived', data = train)

In [ ]:
sns.countplot(x='Pclass',data = train)

In [ ]:
sns.countplot(x='Pclass',data = train,hue = 'Survived')

In [ ]:
sns.pointplot(x='Embarked',data = train, y='Survived')

In [ ]:
sns.countplot(x='Embarked',data = train,hue = 'Pclass')

In [ ]:
grid = sns.FacetGrid(row='Embarked',col='Survived',hue = 'Sex',data = train)
grid.map(sns.histplot, 'Pclass')
grid.add_legend()

In [ ]:
sns.pointplot(x='SibSp',data = train, y='Survived')

In [ ]:
pd.crosstab(train.SibSp,train.Pclass).style.background_gradient()

In [ ]:
sns.pointplot(x='Parch',data = train, y='Survived')

In [ ]:
sns.displot(data= train, x='Fare',bins=15,row='Pclass',hue='Survived')

# Filling the missing values

In [ ]:
test.isnull().sum()

**The age values that are missing can be a tedious task to fill up. It will be a unfair if we fill the age values with the simple imputer technique like mean, median. Relation of age feature with name feature can be handfull for us to derive the age for each missing individual as the name sugggest some terms like Mr. Mrs. Miss, Master**

In [ ]:

for i in train:
    train['Called'] = train.Name.str.extract('([A-Za-z]+)\.')
for i in test:
    test['Called'] = test.Name.str.extract('([A-Za-z]+)\.')

In [ ]:
train['Called'].value_counts()

In [ ]:
train['Called'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)
test['Called'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Dona'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)

In [ ]:
train[['Called','Age']].groupby('Called').mean().sort_values(ascending =True, by = 'Age' )

In [ ]:
test[['Called','Age']].groupby('Called').mean().sort_values(ascending =True, by = 'Age' )

In [ ]:
train.loc[(train.Age.isnull()) & (train.Called == 'Mr'),'Age'] = 33
train.loc[(train.Age.isnull()) & (train.Called == 'Master'),'Age'] = 5
train.loc[(train.Age.isnull()) & (train.Called == 'Miss'),'Age'] = 22
train.loc[(train.Age.isnull()) & (train.Called == 'Mrs'),'Age'] = 36
train.loc[(train.Age.isnull()) & (train.Called == 'Other'),'Age'] = 46

In [ ]:
test.loc[(test.Age.isnull()) & (test.Called == 'Mr'),'Age'] = 33
test.loc[(test.Age.isnull()) & (test.Called == 'Master'),'Age'] = 5
test.loc[(test.Age.isnull()) & (test.Called == 'Miss'),'Age'] = 22
test.loc[(test.Age.isnull()) & (test.Called == 'Mrs'),'Age'] = 36
test.loc[(test.Age.isnull()) & (test.Called == 'Other'),'Age'] = 46

In [ ]:
train['Embarked'].fillna('S',inplace=True)
test['Fare'].fillna(test['Fare'].mean())

# Feature Engineering

Age feature:  
The age feature can make the model complex as this is the continous variable and will have different values. So it will be best idea to normalize or binning them. We can bin them with certain bin width.
bin size = (Maxm_age - Min_age )/5 = 16

In [ ]:
train['A_band'] = 0
train.loc[train['Age']<=16,'A_band'] = 0
train.loc[(train['Age']>16) & (train['Age']<=32) ,'A_band'] = 1
train.loc[(train['Age']>32) & (train['Age']<=48) ,'A_band'] = 2
train.loc[(train['Age']>48) & (train['Age']<=64) ,'A_band'] = 3
train.loc[(train['Age']>64) & (train['Age']<=80) ,'A_band'] = 4
train.head()

In [ ]:
test['A_band'] = 0
test.loc[test['Age']<=16,'A_band'] = 0
test.loc[(test['Age']>16) & (test['Age']<=32) ,'A_band'] = 1
test.loc[(test['Age']>32) & (test['Age']<=48) ,'A_band'] = 2
test.loc[(test['Age']>48) & (test['Age']<=64) ,'A_band'] = 3
test.loc[(test['Age']>64) & (test['Age']<=80) ,'A_band'] = 4

In [ ]:
sns.pointplot(x='A_band',data = train, y='Survived')

In [ ]:
train['Family_Size']=0
train['Family_Size']=train['Parch']+train['SibSp']
train['isAlone']=0
train.loc[train.Family_Size==0,'isAlone']=1

In [ ]:
test['Family_Size']=0
test['Family_Size']=test['Parch']+test['SibSp']
test['isAlone']=0
test.loc[test.Family_Size==0,'isAlone']=1

In [ ]:
sns.pointplot(x='Family_Size',data = train, y='Survived')

In [ ]:
sns.pointplot(x='isAlone',data = train, y='Survived')

In [ ]:
train['Fare_Range'] = pd.qcut(train['Fare'],4)
sns.pointplot(x='Fare_Range',data = train, y='Survived')

In [ ]:
train['F_band'] = 0
train.loc[train['Fare']<=7.91,'F_band'] = 0
train.loc[(train['Fare']>7.91) & (train['Fare']<=14.454) ,'F_band'] = 1
train.loc[(train['Fare']>14.454) & (train['Fare']<=31) ,'F_band'] = 2
train.loc[(train['Fare']>31) & (train['Fare']<=512.39) ,'F_band'] = 3
train.head()

In [ ]:
test['F_band'] = 0
test.loc[test['Fare']<=7.91,'F_band'] = 0
test.loc[(test['Fare']>7.91) & (test['Fare']<=14.454) ,'F_band'] = 1
test.loc[(test['Fare']>14.454) & (test['Fare']<=31) ,'F_band'] = 2
test.loc[(test['Fare']>31) & (test['Fare']<=512.39) ,'F_band'] = 3
test.head()

In [ ]:
sns.pointplot(x='F_band',data = train, y='Survived',hue = "Sex")

Encoding Categorical features 

In [ ]:
train['Sex'].replace(['male','female'],[0,1],inplace=  True)
train['Embarked'].replace(['S','C','Q'],[0,1,2],inplace = True)
train['Called'].replace(['Mr','Mrs','Miss','Master','Other'],[0,1,2,3,4],inplace = True)


In [ ]:
test['Sex'].replace(['male','female'],[0,1],inplace=  True)
test['Embarked'].replace(['S','C','Q'],[0,1,2],inplace = True)
test['Called'].replace(['Mr','Mrs','Miss','Master','Other'],[0,1,2,3,4],inplace = True)

In [ ]:
train.drop(['Name','Age','Ticket','Fare','Cabin','Fare_Range','PassengerId'],axis = 1,inplace=True)

In [ ]:
test.drop(['Name','Age','Ticket','Fare','Cabin','PassengerId'],axis = 1,inplace=True)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
survived = train['Survived']
train1 = train.drop(['Survived'],axis =1)
train1

In [ ]:
survived

# Model Selection

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train1,survived,test_size = 0.3,random_state = 42)

In [ ]:
print(f"X_train:{X_train.shape}\ny_train:{y_train.shape}")
print(f"\nX_test:{X_test.shape}\ny_test:{y_test.shape}")

In [ ]:
logistic_model=LogisticRegression()
logistic_model.fit(X_train,y_train)



In [ ]:
#accuracy for train data
y_pred = logistic_model.predict(X_train)
acc = metrics.accuracy_score(y_train,y_pred)
print(acc)

In [ ]:
#accuracy for test data
y_pred_test = logistic_model.predict(X_test)
acc = metrics.accuracy_score(y_test,y_pred_test)
print(acc)

In [ ]:
cf_matrix = confusion_matrix(y_test,y_pred_test)

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')
ax.set_title('Confusion Matrix \n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

In [ ]:
print(metrics.classification_report(y_test,y_pred_test))

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors = 9)
knn_model.fit(X_train, y_train)
y_pred =knn_model.predict(X_test)
acc = metrics.accuracy_score(y_pred, y_test)
print(acc)

In [ ]:
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
acc= metrics.accuracy_score(y_pred,y_test)
print(acc)

In [ ]:
gb_model = GaussianNB()
gb_model.fit(X_train, y_train)
y_pred = gb_model.predict(X_test)
acc= metrics.accuracy_score(y_pred,y_test)
print(acc)

In [ ]:
svm_model = svm.SVC(kernel = 'rbf', C =1 , gamma = 0.1)
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
acc= metrics.accuracy_score(y_pred,y_test)
print(acc)

In [ ]:
sample_submission

In [ ]:
predicted_values = svm_model.predict(test)

In [ ]:
sample_submission['Survived'] = predicted_values

In [ ]:
sample_submission.to_csv('sample_submission.csv',index = False)